# Objective: To use gguf model locally with langchain 

    embedding: HuggingFaceEmbedding(sentence-transformers/all-mpnet-base-v2)
    
    vector store: Chroma
    
    retriever: from vector store
    
    llm:
        1)mistral_7B_v0.3
        2)capybarahermes-2.5-mistral-7b.Q3_K_L.gguf

## Document Loader
    pdf loader : langchain inbuilt document loader 

In [16]:
from langchain_community.document_loaders import PyPDFLoader
file_path = (r"D:\OneDrive - Adani\Desktop\LEARNING_FOLDER\_Kolkata_2024\1_LLM\3_Text_query_bot\_docs\Leave_Policy_2024.pdf")
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()
len(pages)


4

## Split
    smaller chunks

In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)
len(splits)


15

In [18]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

## Create vector store
    stores embeddings of Documents

In [19]:
from langchain_huggingface import HuggingFaceEmbeddings

In [20]:
new_embeddings = HuggingFaceEmbeddings(model_name= r"D:\OneDrive - Adani\Desktop\LEARNING_FOLDER\_Kolkata_2024\1_LLM\local_downloaded_models\embedding_models\gte-base")
vectorstore = Chroma.from_documents(documents=splits, embedding=new_embeddings)
vectorstore

## Create retriever

In [21]:
retriever = vectorstore.as_retriever(search_type="similarity")

In [22]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

### LLM Used:

## 1)Model: mistral_7B_v0.3
    Langchain + CTransformer
    Size: 4.07 GB 
    RAM usage: how to find??
    Response time:
    
- quantized model with langchain
- on cpu without nvidia gpu
- cpu -- 16 gb RAM
- download the gguf model
- What is quantization --> the higger bit quant --> more ram

In [ ]:
from langchain_community.llms import CTransformers
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

model_dir =  r"D:\OneDrive - Adani\Desktop\LEARNING_FOLDER\_Kolkata_2024\1_LLM\local_downloaded_models\mistral_7B_v0.3"
model_file = "Mistral-7B-Instruct-v0.3.Q4_K_M.gguf"

config = {'context_length': 16000, 'max_new_tokens': 1600}


llm = CTransformers(model= model_dir, model_file = model_file, callbacks=[StreamingStdOutCallbackHandler()], config= config)

## 2)Model:capybarahermes-2.5-mistral-7b.Q3_K_L.gguf

    Langchain + CTransformer
    Size: 3.82 GB 
    RAM usage: 6.02 GB
    Response time:


Model Specs:
- context_length: 32768

def=> number of tokens or words that model takes into account when generating a response

- max_new_tokens: 32768 

def => max number of tokens that can be generated. helps to control the length of generated output
    
(+)Note:
- if less max tokens, the processing will be faster


In [ ]:
from langchain_community.llms import CTransformers
from langchain import PromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

model_dir =  r"D:\OneDrive - Adani\Desktop\LEARNING_FOLDER\_Kolkata_2024\1_LLM\local_downloaded_models"
model_file = "capybarahermes-2.5-mistral-7b.Q3_K_M.gguf"

config = {'context_length': 16000, 'max_new_tokens': 1600}


llm = CTransformers(model= model_dir, model_file = model_file, callbacks=[StreamingStdOutCallbackHandler()], config= config)

## 3) llama-2-7b-chat.Q6_K.gguf
download link: https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/tree/main
Model specifications

In [23]:
from langchain_community.llms import CTransformers
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

model_dir =  r"D:\OneDrive - Adani\Desktop\LEARNING_FOLDER\_Kolkata_2024\1_LLM\local_downloaded_models"
model_file = "llama-2-7b-chat.Q6_K.gguf"

config = {'context_length': 16000, 'max_new_tokens': 1600}


llm = CTransformers(model= model_dir, model_file = model_file, callbacks=[StreamingStdOutCallbackHandler()], config= config)

In [ ]:
llm.invoke("You are a better AI assistant you know")

## Creating Prompt

In [24]:
from langchain.prompts import PromptTemplate

### New Terms:
- Multiqueryretriever: 
    - automates process of tuning
    - to generate multiple queries from different perspective 
    - for each query- returns relevant documents,, takes union across all 
    - Overcomes the limitation of distance based retrieval


In [26]:
from langchain.prompts import ChatPromptTemplate

## Exploring Retrievers

In [27]:
# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [13]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [29]:
retriever = vectorstore.as_retriever()
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

## Chain

In [30]:
chain = (
    {"context": retriever| format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

result=chain.invoke("Give me summary in 5 bullet points only")


Answer: Of course! Based on the provided context, here are five key points summarizing the Leave Policy for Adani Group employees:
1. Employees can avail up to 5.4 days of Special Leave (SL) without any limit.
2. SL can be combined with Paternity Leave/Maternity Leave in case of prolonged illness, subject to submission of medical certificates.
3. Employees have to compulsorily avail at least 15 days of Privilege Leave (PL) in a block of two years for rejuvenation.
4. PL is credited to the employee's Leave account at the end of each Leave Year, and unavailed PL will lapse in the same year.
5. The Sanctioning Authority for leave approval is usually the reporting manager of the employee or any person authorized by the organization.

In [ ]:
result
